In [1]:
# AffectNet has images stored as image format so occlusions are applied
# and stored separately so Keras can use flow_from_directory to train

import os
from scipy import misc
import numpy as np
import re
import imageio
from imutils import face_utils
import dlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import cv2
import skimage
from skimage import img_as_ubyte

def get_patch(c, size):
    top = 0 if c[1]-8 < 0 else c[1]-8
    bot = 47 if c[1]+8 > 47 else c[1]+8
    lef = 0 if c[0]-2 < 0 else c[0]-2
    rig = 47 if c[0]+size-2 > 47 else c[0]+size-2
    return top,bot,lef,rig

def noisy(noise_typ,image,d):
    
    if not d:
        d = {"mouth": np.array([16,38]), "both": np.array([11,18])}
        
    feature = random.choice([0,1])
    if feature == 0:
        top,bot,lef,rig = get_patch(d["both"],40)
    elif feature == 1:
        top,bot,lef,rig = get_patch(d["mouth"],25)
    modded = image.copy()
    row,col = modded.shape
    if noise_typ == "s&p":
        noise = skimage.util.random_noise(modded,mode=noise_typ, amount = 1)
    else:
        noise = skimage.util.random_noise(modded,mode=noise_typ)
    modded[top:bot,lef:rig] = img_as_ubyte(noise[top:bot,lef:rig])
    return (modded.reshape(dim,dim))


emotion = ["angry","disgust","fear","happy","sad","neutral","surprise"]
for em in emotion:
    images = []
    for root, dirnames, filenames in os.walk("../images/AffectNetFlow/train/"+em):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff|JPG|PNG)$", filename):
                filepath = os.path.join(root, filename)
                image = imageio.imread(filepath, as_gray=True)
                image_resized = misc.imresize(image, (48, 48))
                images.append(image_resized)
    images = np.array(images)

    dim = 48
    p = "shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(p)
    for x in range(0,len(images),3):
        gray = images[x].astype('uint8')
        rects = detector(gray, 1)
        features = {}
        # For each detected face, find the landmark.
        for (i, rect) in enumerate(rects):
            # Make the prediction and transfom it to numpy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            left_eye = shape[36]
            right_eye = shape[42]
            mouth = shape[48]
            features = {"mouth": mouth, "both":left_eye}

        gray = cv2.resize(gray, (dim,dim))
        gray = np.array(gray)
        img = noisy("s&p",gray,features).astype('float32')
        images[x] = img

    i=0
    for image in images:
        imageio.imwrite('../images/AffectNetOccluded/train/'+em+'/'+str(i)+'.png', image)
        i+=1